In [1]:
import os
import glob
import cv2
from PIL import Image 
import numpy as np 
import datetime 

import torch
from torchvision.utils import make_grid, save_image
from torch.utils.data import Dataset ,DataLoader
import torchvision.transforms as transforms
import torch.nn.functional as F

from Grad_CAM.img_norm import Normalize, transform_img
from Grad_CAM.model_value import mv

In [2]:
def norm_preprocess(img_pth,means,stds): # img_pth 주면 img -> torch tensor type (1,3,224,224) 
    # means = [r,g,b] / std = [r,g,b]
    p_img = Image.open(img_pth) 
    tensor_img = torch.from_numpy(np.asarray(p_img)).permute(2,0,1).unsqueeze(0).float().div(255).cuda()
    # vgg -> 1,3,224,224
    # alex -> 1,3,227,227
    if tensor_img != (1,3,224,224) :
        tensor_img = F.interpolate(tensor_img,size=(224,224),mode='bilinear', align_corners=False)

    norm_mean_std = Normalize(mean=means, std=stds)
    nt_img = norm_mean_std(tensor_img)

    return nt_img , tensor_img

In [3]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
img_pth = glob.glob('./car_data/test_data/*.jpg')
model_pth = glob.glob('./vgg_pt/*pt')
save_pth = './car_data/grad_cam_result/'
LABEL_NAME = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']
label_idx = 1


In [4]:
for i in model_pth:
    save_name = i.split('\\')[1].split('.')[0]
    save_path = os.path.join(save_pth,save_name)
    # make dir
    if os.path.exists(save_path):
        continue
    else: 
        os.mkdir(save_path)
    

In [5]:
for i in range(0,2):
    mod=model_pth[i]
    models = torch.load(mod).to(DEVICE)
    save_name = mod.split('\\')[1].split('.')[0]
    save_path = os.path.join(save_pth,save_name)

    for v in img_pth:
        name = v.split('/')[2].split('\\')[1]
        img,origin_img = norm_preprocess(v,means=[0.485, 0.456, 0.406], stds=[0.229, 0.224, 0.225])
        grad_cam = mv('conv5',models,img,origin_img,label_idx,name,save_path,LABEL_NAME)
        best_clas,best_clas_name=grad_cam.make_img()
        if label_idx != best_clas: 
            print('-------'+save_name+'--------')
            print(v)
            print(best_clas_name)

            grad_cam = mv('conv5',models,img,origin_img,best_clas,name,save_path,LABEL_NAME,re=True)
            grad_cam.make_img()

        
        
        
        

    

C:\Users\mjy22\AppData\Local\Temp\ipykernel_21556\3694273873.py:4: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_numpy.cpp:178.)
  tensor_img = torch.from_numpy(np.asarray(p_img)).permute(2,0,1).unsqueeze(0).float().div(255).cuda()
c:\Users\mjy22\anaconda3\envs\pytorch\lib\site-packages\torch\nn\modules\module.py:1053: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full

-------vgg16_cifar10_10epoch--------
./car_data/test_data\000851.jpg
truck
-------vgg16_cifar10_10epoch--------
./car_data/test_data\001688.jpg
bird
-------vgg16_cifar10_20epoch--------
./car_data/test_data\000100.jpg
truck
-------vgg16_cifar10_20epoch--------
./car_data/test_data\000143.jpg
truck
-------vgg16_cifar10_20epoch--------
./car_data/test_data\000851.jpg
truck
-------vgg16_cifar10_20epoch--------
./car_data/test_data\001688.jpg
bird


In [6]:
# for v in img_pth:
#     name = v.split('/')[3]
#     img,origin_img = norm_preprocess(v,means=[0.485, 0.456, 0.406], stds=[0.229, 0.224, 0.225])
#     grad_cam = mv('conv5',model_10ep,img,origin_img,name,save_path)
#     grad_cam.make_img()
    
    